In [1]:
%matplotlib inline
from fns import *
from syntheticdata import synthetic_generation
# import ipyvolume

In [2]:
import plotly
plotly.tools.set_credentials_file(username='gpernelle', api_key='4KIdNJBBApIeebdKxyN9')


## setting up the unet

## training

In [ ]:
trainer = Trainer(batch_size=3, optimizer="adam", filter_size=3, layers=4)
path = trainer.train("./models/unet_trained_mix_dropout-025",
                     synth=1,
                     freeze_deep_layers=0,
                     training_array = None,
                     validation_array = None,
                     testing_array = None,
                     training_iters=415, 
                     epochs=100, 
                     dropout=0.5, 
                     restore=True,
                     display_step=1)

100%|██████████| 16/16 [00:01<00:00, 11.32it/s]


Feature shape: (415, 148, 148, 148, 1)
b'\n%/job:localhost/replica:0/task:0/cpu:0\x12\tlocalhost\x1a\x0b_0_Iterator \xa4\xd6\xcb\xd1\xa6\xeb\xb2\xc1\xc7\x01*.N10tensorflow12_GLOBAL__N_116IteratorResourceE'
b'\n%/job:localhost/replica:0/task:0/cpu:0\x12\tlocalhost\x1a\x0b_0_Iterator \xa4\xd6\xcb\xd1\xa6\xeb\xb2\xc1\xc7\x01*.N10tensorflow12_GLOBAL__N_116IteratorResourceE'
(tf.float32, tf.float32)


In [4]:
# with tf.Session() as sess:
#     saver = tf.train.Saver()
#     ckpt = tf.train.get_checkpoint_state('./unet_trained')
#     if ckpt and ckpt.model_checkpoint_path:
#         saver.restore(sess, ckpt.model_checkpoint_path)
        
#     tvars = tf.trainable_variables()
#     layer_down_0 = [var for var in tvars if 'layer_down_0' in var.name]
#     for var in layer_down_0:
#         print(var)

### Predict

In [14]:
case=72
image_name = '/mnt/DATA/gp1514/Dropbox/2016-paolo/preprocessed_data/LabelMapsNEW2_1.00-1.00-1.00/0%d/case.nrrd'%case
label_name = '/mnt/DATA/gp1514/Dropbox/2016-paolo/preprocessed_data/LabelMapsNEW2_1.00-1.00-1.00/0%d/needles.nrrd'%case

In [14]:
image_name = '/mnt/DATA/gp1514/Dropbox/2016-paolo/preprocessed_data/LabelMapsNEW2_1.00-1.00-1.00_synthetic_occulted/nrrd/training_case20.nrrd'
label_name = '/mnt/DATA/gp1514/Dropbox/2016-paolo/preprocessed_data/LabelMapsNEW2_1.00-1.00-1.00_synthetic_occulted/nrrd/training_labelmap20.nrrd'

In [25]:
data, options = nrrd.read(image_name)
label_data, options = nrrd.read(label_name)

In [29]:
data = data.astype(np.float32)
# prediction = predict(data, "./models/unet_trained_mix_minmax_l4_f3")
# prediction = predict(data, "./models/unet_trained_mix_dropout-025")
prediction = predict(data, "./models/unet_trained_bk_2")

--------------------------------------------------
Loading and Preparing the data
Original input shape (300, 195, 192)
Padded input shape: (388, 283, 280)
# of parts 7 7 5
number of tiles: 96 
data shape (96, 148, 148, 148)


 10%|█         | 10/96 [00:00<00:00, 93.61it/s]

--------------------------------------------------
Starting the segmenter.


100%|██████████| 96/96 [00:01<00:00, 93.76it/s]


Feature shape:  (96, 148, 148, 148, 1)
INFO:tensorflow:Restoring parameters from ./models/unet_trained_bk_2/model 65.cpkt


100%|██████████| 96/96 [00:10<00:00,  9.50it/s]


prediction shape: (96, 1, 60, 60, 60, 1)
(300, 195, 192) (96, 148, 148, 148)
Merging subvolumes
--------------------------------------------------
Padded input shape: (388, 283, 280)
# of parts 7 7 5
# of subvolumes merged:  96
(array([0, 1, 2, 3, 4, 5, 6, 7], dtype=uint16), array([11186129,    37819,     7570,      348,      127,        4,
              2,        1]))
(300, 195, 192)


In [30]:
def dice(logits, labels):
        flat_logits = logits.flatten()
        flat_labels = labels.flatten()
        intersection = np.sum(flat_logits*flat_labels)
        union = np.sum(flat_logits) + np.sum(flat_labels)
        loss = 1 - 2 * intersection / union
        return loss

In [31]:
dice(prediction, label_data)

0.61717780127821964

In [32]:
islands = post_processing(prediction, min_area=int(200), max_residual=float(2))
islands.shape

Number of regions: 3768
1 1.00690668576 236
2 1.35952475755 402
3 1.38733703236 309
4 1.17615834233 394
5 1.13228116317 817
6 1.08189981065 216
7 1.32486760665 517
8 1.08249467432 346
9 1.89892639071 442
10 1.14771430718 514
11 1.1640663427 630
12 1.98927214603 961
13 1.15825292699 410
14 1.16590763864 458
15 1.10359705092 398
16 1.03556442425 244
17 1.17355098432 401
18 1.13819161118 363
19 1.10021092882 515
20 1.13426083234 416
21 1.07052842857 300
22 1.1993428943 225
23 1.16310827731 517
24 1.64365072697 279
25 1.14753776442 467
26 1.13588867453 399
27 1.20630827002 580
28 1.37647455234 444
29 1.12918674261 239
30 1.15509080492 692
31 1.15529849421 793
32 1.10289474571 338
33 1.1469051261 843
34 1.16238944516 410
35 1.17854649003 513
36 1.27056281542 332


(300, 195, 192)

In [33]:
# %matplotlib notebook
# xs,ys,zs = np.where(prediction == 1)

# fig = plt.figure(figsize=(6,6))
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(xs, ys, zs, marker='o', alpha=0.1, s=1)
# plt.show()

# # fig = plt.figure(figsize=(6,6))
# # ax = fig.add_subplot(111, projection='3d')
# xs,ys,zs = np.where(label_data == 1)
# ax.scatter(xs, ys, zs, marker='o',color='g', alpha=0.1, s=5)
# plt.show()

In [34]:
import plotly.plotly as py
import plotly.graph_objs as go
x,y,z = np.where(label_data == 1)
xs,ys,zs = np.where(islands != 0)

trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=3,
        line=dict(
            color='green',
            width=2
        ),
        opacity=0.3
    )
)

trace2 = go.Scatter3d(
    x=xs,
    y=ys,
    z=zs,
    mode='markers',
    marker=dict(
        color='red',
        size=3,
        symbol='circle',
        line=dict(
            color='rgb(204, 204, 204)',
            width=1
        ),
        opacity=0.3
    )
)
data = [trace1, trace2]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='simple-3d-scatter')

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~gpernelle/0 or inside your plot.ly account where it is named 'simple-3d-scatter'


In [48]:
import ipyvolume.pylab as p3
p3.figure(figsize=(15,15))
ipyvolume.quickvolshow(islands, opacity=1)
ipyvolume.quickvolshow(label_data, opacity=1)

/mnt/DATA/gp1514/.pyenv/versions/anaconda3-4.4.0/envs/needles/lib/python3.6/site-packages/ipyvolume/serialize.py:29: RuntimeWarning: invalid value encountered in true_divide
  gradient = gradient / np.sqrt(gradient[0]**2 + gradient[1]**2 + gradient[2]**2)
/mnt/DATA/gp1514/.pyenv/versions/anaconda3-4.4.0/envs/needles/lib/python3.6/site-packages/ipyvolume/serialize.py:29: RuntimeWarning: invalid value encountered in true_divide
  gradient = gradient / np.sqrt(gradient[0]**2 + gradient[1]**2 + gradient[2]**2)


In [35]:
x,y,z = np.where(label_data == 1)
ipyvolume.quickscatter(x,y,z, size=0.5, marker="sphere", opacity=0.2)

x,y,z = np.where(islands == 1)
ipyvolume.quickscatter(x,y,z, size=0.5, marker="sphere", opacity=0.2, color='blue')

In [ ]:
np.random.seed(1)
islands = measure.label(prediction)
K = np.max(islands)
cp =sns.color_palette("Set2", K)
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111, projection='3d')
for j in range(1,K):
    xs,ys,zs = np.where(islands == j)
    ax.scatter(xs, ys, zs, marker='o',color=cp[j], alpha=0.9, s=2)
plt.show()

In [ ]:
np.random.seed(1)
islands = measure.label(label_data[i,...,0][44:-44,44:-44,44:-44])
K = np.max(islands)
cp =sns.color_palette("Set2", K)
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111, projection='3d')
for j in range(1,K):
    xs,ys,zs = np.where(islands == j)
    ax.scatter(xs, ys, zs, marker='o',color=cp[j], alpha=0.9, s=2)
plt.show()

In [ ]:
image_name = '/mnt/DATA/gp1514/Dropbox/2016-paolo/preprocessed_data/LabelMapsNEW2_1.00-1.00-1.00/075/case.nrrd'
label_name = '/mnt/DATA/gp1514/Dropbox/2016-paolo/preprocessed_data/LabelMapsNEW2_1.00-1.00-1.00/075/needles.nrrd'

In [ ]:
# print(img.shape)
# print(data.shape)

In [ ]:
# data.shape

In [ ]:
tiles = (148,148,148)
tile = 148

In [ ]:
data, options = nrrd.read(image_name)
data = data.astype(np.float32)
print(data.shape)
d = data.resize(max(data.shape[0],tile),
               max(data.shape[1],tile),
               max(data.shape[2],tile))
print(data.shape)
print(options)

## Inference pipeline

In [ ]:
arr_data = cutVolume(data)
arr_pred = predict_full_volume(net, arr_data, model_path="./unet_trained/model 99.cpkt")
full_pred = recombine(arr_pred, data)

In [ ]:
def post_processing(full_pred, min_area=150, max_residual=10):
    ''' Clustering + removing small clusters + keeping only line-looking clusters'''
    islands_ = measure.label(full_pred)
    regions = measure.regionprops(islands_)
    islands = np.zeros_like(full_pred, dtype=np.uint8)
    K = len(regions)
    print('Number of regions: %d' % K)
    i=0
    for k in range(K):
        region = regions[k]
        coords = region.coords
        if region.area > min_area:
            lm = measure.LineModelND()
            lm.estimate(coords)
            res = lm.residuals(coords)
            mean_res = np.mean(res)
            if mean_res < max_residual:
                i+=1
                print(k, i, mean_res, np.std(res), region.area)
                for x,y,z in coords:
                    islands[x,y,z] = i


In [ ]:
islands = post_processing(full_pred)

In [ ]:
islands_ = measure.label(full_pred)
regions = measure.regionprops(islands_)
region = regions[691]
lm = measure.LineModelND()
lm.estimate(region.coords)
res = lm.residuals(region.coords)
res
K = np.max(islands)
cp = sns.color_palette("Set2", K)
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111, projection='3d')

xs,ys,zs = region.coords.T
ax.scatter(xs, ys, zs, marker='o',color=cp[j], alpha=0.9, s=2)
plt.show()

In [ ]:
print(np.unique(islands, return_counts=True))

In [ ]:
# islands = measure.label(full_pred)
K = np.max(islands)
cp = sns.color_palette("Set2", K)
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111, projection='3d')
for j in range(1,K):
    xs,ys,zs = np.where(islands == j)
    ax.scatter(xs, ys, zs, marker='o',color=cp[j], alpha=0.9, s=2)
plt.show()

In [ ]:
nrrd.write('test75.nrrd', islands, options=options)

In [ ]:
islands.shape
print(np.unique(islands, return_counts=True))

In [ ]:
image_name = '/mnt/DATA/gp1514/Dropbox/2016-paolo/preprocessed_data/LabelMapsNEW2_1.00-1.00-1.00/074/needles.nrrd'
data, options = nrrd.read(image_name)
data = data.astype(np.int8)

In [ ]:
arr_labels = cutVolume(data)

In [ ]:
# '''
# EXPORT
# '''

# with tf.Graph().as_default():
#         # Inject placeholder into the graph
#         serialized_tf_example = tf.placeholder(tf.string, name='input_image')
#         feature_configs = {'data': tf.FixedLenFeature(shape=[1,148,148,148,1], dtype=tf.float32),}
#         tf_example = tf.parse_example(serialized_tf_example, feature_configs)
#         x_test = tf_example['data']
#         # now the image shape is (1,148,148,148,1)

#         # Create UNET model
#         net = Unet(channels=1, n_class=1, layers=4, pool_size=2, features_root=16, summaries=True)
#         # Create saver to restore from checkpoints
#         saver = tf.train.Saver()

#         with tf.Session() as sess:
            
#             # Restore the model from last checkpoints
#             saver.restore(sess, "./unet_trained/model 99.cpkt")
            
            
#             y_dummy = np.empty((1,148,148,148,1))

#             # (re-)create export directory
#             export_path = './export/'
#             if os.path.exists(export_path):
#                 shutil.rmtree(export_path)

#             # create model builder
#             builder = tf.saved_model.builder.SavedModelBuilder(export_path)

#             # create tensors info
#             predict_tensor_inputs_info = tf.saved_model.utils.build_tensor_info(x_test)
#             predict_tensor_scores_info = tf.saved_model.utils.build_tensor_info(net.predicter_label)

#             # build prediction signature
#             prediction_signature = (
#                 tf.saved_model.signature_def_utils.build_signature_def(
#                     inputs={'images': predict_tensor_inputs_info},
#                     outputs={'scores': predict_tensor_scores_info},
#                     method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME
#                 )
#             )

#             # save the model
#             legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')
#             builder.add_meta_graph_and_variables(
#                 sess, [tf.saved_model.tag_constants.SERVING],
#                 signature_def_map={
#                     'predict_images': prediction_signature
#                 },
#                 legacy_init_op=legacy_init_op)

#             builder.save()

# print("Successfully exported UNET model")